In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Project_NLP/dataset/HateSpeechDataset.csv') #change this to your own path in drive
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Project_NLP/dataset/HateSpeechDataset.csv'

In [ ]:

df = df.drop(['Content_int'], axis=1)


In [ ]:
df.shape

(440906, 2)

In [ ]:
df['Label'].value_counts()

,count
Label,
0,361594
1,79305
Label,7


In [ ]:
df.head()

,Content,Label
0,denial of normal the con be asked to comment o...,1
1,just by being able to tweet this insufferable ...,1
2,that is retarded you too cute to be single tha...,1
3,thought of a real badass mongol style declarat...,1
4,afro american basho,1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# These list are some of the common words used in hate speech for each columns
racism = ["nigga", "monkey", "chimp", "gook", "wetback", "chink", "cracker", "coon","savage", "redskin", "paki" ,"spic", "beaner", "raghead", "kaffir" ,"wog", "slant-eye", "ape", "coonass"]
homophobia = ["faggot", "dyke", "tranny", "queer", "gay", "lesbian", "bisexual", "homosexual", "transgender", "LGBTQ", "shemale", "trannie"]
sexism = ["slut", "whore", "bitch", "cunt", "pussy", "dick", "manwhore", "feminazi", "golddigger", "hoe", "skank", "milf", "cuntbag"]
xenophobia = ["immigrant", "refugee", "foreigner", "alien", "outsider", "illegal", "terrorist", "muslim", "arab", "asian", "black", "mexican", "african", "sand nigger", "raghead", "towelhead"]

In [ ]:

def categorize_hate_speech(row):
  if row['Label'] == "0":
    return 'Not Hate'
  else:
    content = row['Content'].lower()
    if any(word in content for word in racism):
      return 'racism'
    elif any(word in content for word in homophobia):
      return 'homophobia'
    elif any(word in content for word in sexism):
      return 'sexism'
    elif any(word in content for word in xenophobia):
      return 'xenophobia'
    else:
      return 'others'

df['category'] = df.apply(categorize_hate_speech, axis=1)


In [ ]:
df.head()

,Content,Label,category
0,denial of normal the con be asked to comment o...,1,others
1,just by being able to tweet this insufferable ...,1,others
2,that is retarded you too cute to be single tha...,1,others
3,thought of a real badass mongol style declarat...,1,others
4,afro american basho,1,others


In [ ]:
if 'category' in df.columns:
    df_filtered = df[df['category'] != 'others']
else:
    print("Column 'category' does not exist in the DataFrame")

In [ ]:
df_filtered.head()

,Content,Label,category
7,the dead what a slut still warm when she tweet...,1,sexism
8,let your tweets be harmless it will not affect...,1,homophobia
10,i feel so much secondhand embarrassment when a...,1,racism
15,this fucking faggot,1,homophobia
16,holy shit what a faggot laughing my ass off,1,homophobia


In [ ]:
df_filtered.shape

(389240, 3)

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt_tab")
def preprocess(text):
    # Remove non-alphabet characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()

    # Remove punctuation
    punctuationfree = "".join([i for i in text if i not in string.punctuation])

    # Tokenize the text
    tokenized = nltk.word_tokenize(punctuationfree)

    #  Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in tokenized if word not in stop_words]

    # Join the filtered words with ' ' as the separator
    return " ".join(filtered_words)

df_filtered['processed_content'] = df_filtered['Content'].apply(preprocess)
df_filtered

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
<ipython-input-9-2f1973104961>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['processed_content'] = df_filtered['Content'].apply(preprocess)


,Content,Label,category,processed_content
7,the dead what a slut still warm when she tweet...,1,sexism,dead slut still warm tweeted slut vagina mate ...
8,let your tweets be harmless it will not affect...,1,homophobia,let tweets harmless affect way faggot one n
10,i feel so much secondhand embarrassment when a...,1,racism,feel much secondhand embarrassment white perso...
15,this fucking faggot,1,homophobia,fucking faggot
16,holy shit what a faggot laughing my ass off,1,homophobia,holy shit faggot laughing ass
...,...,...,...,...
440901,crash another movie from left field i have to ...,0,Not Hate,crash another movie left field ask star wars e...
440902,i why do not you debate first before starting ...,0,Not Hate,debate first starting edit war
440903,removal of i reverted the removal of the above...,0,Not Hate,removal reverted removal section believe misgu...
440904,i have unblocked you eddie as i discussed on u...,0,Not Hate,unblocked eddie discussed user talkable accusi...


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = df_filtered['processed_content']
y = df_filtered['category']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

max_length = 100
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

train_padded


array([[   1, 3238,    1, ...,    0,    0,    0],
       [ 102,  109,  571, ...,    0,    0,    0],
       [4531,  155,   52, ...,    0,    0,    0],
       ...,
       [   1,    1,   14, ...,    0,    0,    0],
       [ 376,   43, 1497, ...,    0,    0,    0],
       [ 103,  200,    0, ...,    0,    0,    0]], dtype=int32)

In [ ]:
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# prompt: load the lstm model and make prediction

from tensorflow.keras.models import load_model

# Load the saved LSTM model
model = load_model('/content/drive/MyDrive/Project_NLP/lstm_model.keras')
# Example prediction for a new text
new_text = "asdf asd."
new_text = preprocess(new_text)  # Preprocess the new text using your preprocessing function
print(type(new_text))
new_sequence = tokenizer.texts_to_sequences([new_text])
print(new_sequence)
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')
prediction = model.predict(new_padded)

# Decode the prediction to get the category
predicted_label = label_encoder.inverse_transform([prediction.argmax()])[0]
print(f"Predicted category: {predicted_label}")


<class 'str'>
[[1, 1]]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
Predicted category: Not Hate


## LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

embedding_dim = 128
model = Sequential()
model.add(Embedding(5000, embedding_dim, input_length=max_length))
model.add(LSTM(64))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build((None, max_length))
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 689,733 (2.63 MB)

 Trainable params: 689,733 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
num_epochs = 10
history = model.fit(train_padded, y_train, epochs=num_epochs, validation_data=(test_padded, y_test))

Epoch 1/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 80s 8ms/step - accuracy: 0.9298 - loss: 0.3027 - val_accuracy: 0.9441 - val_loss: 0.1600
Epoch 2/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 80s 8ms/step - accuracy: 0.9483 - loss: 0.1438 - val_accuracy: 0.9498 - val_loss: 0.1384
Epoch 3/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 85s 8ms/step - accuracy: 0.9554 - loss: 0.1229 - val_accuracy: 0.9490 - val_loss: 0.1396
Epoch 4/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 83s 9ms/step - accuracy: 0.9598 - loss: 0.1104 - val_accuracy: 0.9504 - val_loss: 0.1400
Epoch 5/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 136s 8ms/step - accuracy: 0.9634 - loss: 0.1007 - val_accuracy: 0.9501 - val_loss: 0.1467
Epoch 6/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 75s 8ms/step - accuracy: 0.9681 - loss: 0.0907 - val_accuracy: 0.9499 - val_loss: 0.1494
Epoch 7/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 76s 8ms/step - accuracy: 0.9711 - loss: 0.0825 - val_accuracy: 0.9496 - val_loss: 0.1562
Epoch 8/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 85s 8ms/step - accuracy: 0.9748 - loss: 

In [ ]:
# prompt: print the classification report and accuracy score

from sklearn.metrics import classification_report, accuracy_score

y_pred = model.predict(test_padded)
y_pred_classes = [label_encoder.classes_[i] for i in y_pred.argmax(axis=1)]
y_test_classes = [label_encoder.classes_[i] for i in y_test]


print(classification_report(y_test_classes, y_pred_classes))
print("Accuracy:", accuracy_score(y_test_classes, y_pred_classes))

2433/2433 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step
              precision    recall  f1-score   support

    Not Hate       0.97      0.98      0.97     72447
  homophobia       0.75      0.66      0.70       886
      racism       0.49      0.33      0.39       800
      sexism       0.57      0.43      0.49      1905
  xenophobia       0.65      0.70      0.68      1810

    accuracy                           0.95     77848
   macro avg       0.69      0.62      0.65     77848
weighted avg       0.94      0.95      0.94     77848

Accuracy: 0.9465882232041928


In [ ]:
_, accuracy = model.evaluate(test_padded, y_test)
print('Accuracy: {}'.format(accuracy))


2433/2433 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9473 - loss: 0.1769
Accuracy: 0.9465882182121277


In [ ]:

model.save('/content/drive/MyDrive/Project_NLP/lstm_model.keras')

## GRU

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

# Define parameters
embedding_dim = 128
vocab_size = 5000
max_length = 100    # Length of padded sequences
num_classes = len(label_encoder.classes_)  # Ensure label_encoder is fitted

# Build the GRU model
gru_model = Sequential()
gru_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
gru_model.add(GRU(64))
gru_model.add(Dense(num_classes, activation='softmax'))  # Use softmax for multi-class classification

# Compile the model
gru_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Explicitly build the model to ensure it's initialized
gru_model.build((None, max_length))# Batch size is None, indicating any number of samples

gru_model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 64)                  │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 677,573 (2.58 MB)

 Trainable params: 677,573 (2.58 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
num_epochs = 10
gru_history = gru_model.fit(train_padded, y_train, epochs=num_epochs, validation_data=(test_padded, y_test))


Epoch 1/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 71s 7ms/step - accuracy: 0.9356 - loss: 0.2471 - val_accuracy: 0.9477 - val_loss: 0.1390
Epoch 2/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 67s 7ms/step - accuracy: 0.9527 - loss: 0.1270 - val_accuracy: 0.9518 - val_loss: 0.1315
Epoch 3/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 82s 7ms/step - accuracy: 0.9586 - loss: 0.1104 - val_accuracy: 0.9517 - val_loss: 0.1326
Epoch 4/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 82s 7ms/step - accuracy: 0.9644 - loss: 0.0974 - val_accuracy: 0.9507 - val_loss: 0.1392
Epoch 5/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 67s 7ms/step - accuracy: 0.9692 - loss: 0.0849 - val_accuracy: 0.9504 - val_loss: 0.1461
Epoch 6/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 82s 7ms/step - accuracy: 0.9738 - loss: 0.0733 - val_accuracy: 0.9486 - val_loss: 0.1579
Epoch 7/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 82s 7ms/step - accuracy: 0.9782 - loss: 0.0623 - val_accuracy: 0.9496 - val_loss: 0.1697
Epoch 8/10
9731/9731 ━━━━━━━━━━━━━━━━━━━━ 82s 7ms/step - accuracy: 0.9806 - loss: 0

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = gru_model.predict(test_padded)
y_pred_classes = [label_encoder.classes_[i] for i in y_pred.argmax(axis=1)]
y_test_classes = [label_encoder.classes_[i] for i in y_test]


print(classification_report(y_test_classes, y_pred_classes))

2433/2433 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
              precision    recall  f1-score   support

    Not Hate       0.97      0.98      0.97     72447
  homophobia       0.72      0.63      0.67       886
      racism       0.55      0.40      0.46       800
      sexism       0.58      0.49      0.53      1905
  xenophobia       0.68      0.63      0.65      1810

    accuracy                           0.95     77848
   macro avg       0.70      0.62      0.66     77848
weighted avg       0.94      0.95      0.95     77848



In [ ]:
_, gru_accuracy = gru_model.evaluate(test_padded, y_test)
print('GRU Accuracy: {}'.format(gru_accuracy))


2433/2433 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9449 - loss: 0.1976
GRU Accuracy: 0.9448797702789307


In [ ]:
new_text = ["you are a stupid nigger"]
new_sequences = tokenizer.texts_to_sequences(new_text)
new_padded = pad_sequences(new_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = gru_model.predict(new_padded)
predicted_labels = [label_encoder.classes_[i] for i in predictions.argmax(axis=1)]


print("Predicted Labels:", predicted_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Predicted Labels: ['racism']


In [ ]:
gru_model.save('/content/drive/MyDrive/Project_NLP/gru_model.keras')